In [43]:
%matplotlib inline
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from scipy import stats
from ast import literal_eval # 문자열 모형의 딕트를 스근하게 딕트로 바꾸어 준다. 
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from surprise import Reader, Dataset, SVD, evaluate

In [63]:
dataSet = pd.read_csv('crawler/top.csv')
dataSet = dataSet.drop(columns='Unnamed: 0', axis=1)

In [64]:
name_popId = dataSet[['name', 'cosmeticId']]

In [65]:
name_popId = name_popId.drop_duplicates().set_index('name')

In [66]:
name_popId

,cosmeticId
name,
비타민 워터리 선 젤 SPF50+ PA++++,0
퓨어 미네랄 선크림 SPF50+ PA++++,1
썬 프로텍트 워터젤 SPF50 PA+++,2
산소수 라이트 선 젤 SPF50+ PA++++,3
유브이 퍼펙션 에브리데이썬 SPF50+ PA++++,4
수분 촉촉 데일리 선 로션 SPF30 PA++,5
UV 퍼펙트 에어쿨 선 젤 SPF50+ PA++++,6
유비데아 XL 멜트-인 내츄럴 크림 SPF50 PA++++,7
유브이 더블 컷 산뜻 선 젤 SPF50+ PA++++,8


In [ ]:
dataSet = dataSet.drop('name', axis=1)

In [51]:
dataSet.head(5)

,cosmeticId,rate,review,type
0,0,4.33,일본 비오레 아쿠아 썬크림 다 써서 최대한 비슷한거 찾아봤더니 이 제품이 제일 비슷...,건성
1,0,4.33,글픽보고 세일때 샀는데 끈적임없고 흡수가 빨랐어요!! 향도 레몬?비타민?같은 향이라...,건성
2,0,4.33,우오 나 이거 3위일때인가 샀는데 지금 1위네영?\r\n가지고 다니던 썬크림을 잃어...,건성
3,0,4.33,일단 제형이 수분감 있어서 좋구 백탁없구 쓰기 너무 좋아요 하지만..저에겐 눈시림이...,건성
4,0,4.50,끈적임없고 세미매트한 마무리감 톤업이 되지않는 제품이라서 팔에 바르기가 좋고 메이크...,지성


In [53]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words=['\r', '\n'])
tf_matrix = tf.fit_transform(dataSet['review'])

In [54]:
cosine_sim = linear_kernel(tf_matrix, tf_matrix)
cosine_sim[0]

array([1.        , 0.02124663, 0.0331748 , ..., 0.012849  , 0.        ,
       0.        ])

In [55]:
reader = Reader()

In [56]:
dataSet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1923 entries, 0 to 1922
Data columns (total 4 columns):
cosmeticId    1923 non-null int64
rate          1923 non-null float64
review        1923 non-null object
type          1923 non-null object
dtypes: float64(1), int64(1), object(2)
memory usage: 60.2+ KB


In [57]:
type = {'건성': 0, '지성': 1, '중성': 2, '복합성': 3, '민감성': 4}
dataSet['type'] = dataSet['type'].map(type)
data = Dataset.load_from_df(dataSet[['cosmeticId', 'type', 'rate']] , reader)

In [58]:
data.split(n_folds=5)

In [59]:
svd = SVD()
evaluate(svd, data, measures=['RMSE', 'MAE'])

C:\Users\LeeSangHoon\Anaconda3\envs\tensorflow\lib\site-packages\surprise\evaluate.py:66: UserWarning: The evaluate() method is deprecated. Please use model_selection.cross_validate() instead.
  'model_selection.cross_validate() instead.', UserWarning)
C:\Users\LeeSangHoon\Anaconda3\envs\tensorflow\lib\site-packages\surprise\dataset.py:193: UserWarning: Using data.split() or using load_from_folds() without using a CV iterator is now deprecated. 
  UserWarning)


Evaluating RMSE, MAE of algorithm SVD.

------------
Fold 1


RMSE: 0.1915
MAE:  0.1259
------------
Fold 2
RMSE: 0.1810
MAE:  0.1359
------------
Fold 3


RMSE: 0.2126
MAE:  0.1448
------------
Fold 4
RMSE: 0.1662
MAE:  0.1288
------------
Fold 5


RMSE: 0.1909
MAE:  0.1382
------------
------------
Mean RMSE: 0.1884
Mean MAE : 0.1347
------------
------------


CaseInsensitiveDefaultDict(list,
                           {'rmse': [0.19154490904663232,
                             0.180973269974746,
                             0.21257195327249104,
                             0.16615493791332192,
                             0.1908593439817688],
                            'mae': [0.12589156539762894,
                             0.1358954673364064,
                             0.14477991124295256,
                             0.1288424435094133,
                             0.13818635229040205]})

In [60]:
trainset = data.build_full_trainset()
svd.train(trainset)

C:\Users\LeeSangHoon\Anaconda3\envs\tensorflow\lib\site-packages\surprise\prediction_algorithms\algo_base.py:51: UserWarning: train() is deprecated. Use fit() instead
  warnings.warn('train() is deprecated. Use fit() instead', UserWarning)


In [62]:
dataSet.head(5)

,cosmeticId,rate,review,type
0,0,4.33,일본 비오레 아쿠아 썬크림 다 써서 최대한 비슷한거 찾아봤더니 이 제품이 제일 비슷...,0
1,0,4.33,글픽보고 세일때 샀는데 끈적임없고 흡수가 빨랐어요!! 향도 레몬?비타민?같은 향이라...,0
2,0,4.33,우오 나 이거 3위일때인가 샀는데 지금 1위네영?\r\n가지고 다니던 썬크림을 잃어...,0
3,0,4.33,일단 제형이 수분감 있어서 좋구 백탁없구 쓰기 너무 좋아요 하지만..저에겐 눈시림이...,0
4,0,4.50,끈적임없고 세미매트한 마무리감 톤업이 되지않는 제품이라서 팔에 바르기가 좋고 메이크...,1


In [67]:
def hybrid(type, cosmeticsNum=0):
    print(type, cosmeticsNum)

In [68]:
hybrid(0, 2)

0 2


In [72]:
indices = pd.Series(dataSet.index, index=dataSet['type'])

In [73]:
indices

type
건성        0
건성        1
건성        2
건성        3
지성        4
지성        5
지성        6
지성        7
중성        8
중성        9
중성       10
중성       11
복합성      12
복합성      13
복합성      14
복합성      15
민감성      16
민감성      17
민감성      18
민감성      19
건성       20
건성       21
건성       22
건성       23
지성       24
지성       25
지성       26
지성       27
중성       28
중성       29
       ... 
중성     1893
중성     1894
복합성    1895
복합성    1896
복합성    1897
복합성    1898
민감성    1899
민감성    1900
민감성    1901
민감성    1902
건성     1903
건성     1904
건성     1905
건성     1906
지성     1907
지성     1908
지성     1909
지성     1910
중성     1911
중성     1912
중성     1913
중성     1914
복합성    1915
복합성    1916
복합성    1917
복합성    1918
민감성    1919
민감성    1920
민감성    1921
민감성    1922
Length: 1923, dtype: int64